This guide shows practical patterns to compute expectation values of the form

$⟨𝑂⟩_𝜓=⟨ψ∣O∣ψ⟩$

using ffsim, where
𝑂 can be a Hamiltonian or any other operator supported by `ffsim.linear_operator`.

We cover:

expectation values from a state vector,

the fast path when an operator provides a diagonal representation,

expectation values for operators given as a FermionOperator.

We again use:

`norb`: number of spatial orbitals

`nelec = (n_alpha, n_beta)`: number of alpha, beta electrons

𝜓 is a state vector in the corresponding Fock basis sector

In ffsim, the dimension of the sector can be found via:

`ffsim.dim(norb, nelec)`.

In [7]:
import numpy as np
import ffsim

rng = np.random.default_rng(0)

norb = 4
nelec = (2, 2)
dim = ffsim.dim(norb, nelec)

# A random normalized complex state vector |psi>
psi = ffsim.random.random_state_vector(dim, seed=rng)
psi = psi / np.linalg.norm(psi)

print(dim, np.linalg.norm(psi))


36 1.0000000000000002


**Method 1: Use a LinearOperator**

Most operators in ffsim can be converted to a `scipy.sparse.linalg.LinearOperator` using:

linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)

Then compute

$⟨𝑂⟩_𝜓=⟨ψ∣(Oψ)⟩$

with np.vdot, which performs the conjugate dot product.

In [15]:
# Example operator: DiagonalCoulombHamiltonian (chosen because it's common and supports diag())
one_body_tensor = np.diag(np.random.rand(norb))
diag_coulomb_mat = ffsim.random.random_real_symmetric_matrix(norb, seed=rng, dtype=float)
diag_coulomb_mats = np.stack([diag_coulomb_mat, diag_coulomb_mat])
H = ffsim.DiagonalCoulombHamiltonian(one_body_tensor, diag_coulomb_mats, constant=0.0)

H_linop = ffsim.linear_operator(H, norb=norb, nelec=nelec)

expval_linop = np.vdot(psi, H_linop @ psi)
print(expval_linop)


(22.585543465564925+0j)


**Notes and pitfalls**

Prefer `np.vdot(psi, ...)` over `psi.conj().T @ ...` for clarity and correct complex conjugation.

For numerical stability, ensure `psi` is normalized (as done above).

For Hermitian operators, the expectation value should be real up to numerical error; you can take `expval.real`.

In [16]:
# For a Hermitian operator, the imaginary part should be near zero (numerical noise only).
print(float(expval_linop.real), float(abs(expval_linop.imag)))

22.585543465564925 0.0


**Method 2 (fast path): Use the diagonal when available**


Some operators support an efficient diagonal extraction:

diag = ffsim.diag(op, norb=norb, nelec=nelec)

If you have the diagonal 𝑑 of 𝑂 in the chosen basis, then

$⟨𝑂⟩_𝜓=\sum_k 𝑑_𝑘 ∣𝜓_𝑘∣^2$.

This avoids forming `O @ psi` and is typically faster.

In [17]:
diag = ffsim.diag(H, norb=norb, nelec=nelec)
diag = np.array(diag, dtype=psi.dtype)
expval_diag = np.vdot(psi, diag * psi)  # elementwise multiply
print(expval_diag)

(22.585543465564925-2.220446049250313e-16j)


In [18]:
np.testing.assert_allclose(expval_diag, expval_linop, rtol=1e-10, atol=1e-10)
"Diagonal method matches LinearOperator method."

'Diagonal method matches LinearOperator method.'

Method 3: Start from a FermionOperator

If your operator is expressed as a `FermionOperator`, you can compute expectation values the same way:

Build a `FermionOperator` (e.g., from a Hamiltonian object), then

Convert it to a linear operator and compute $⟨ψ∣O∣ψ⟩$.

In [19]:
op = ffsim.fermion_operator(H)
print(op)

FermionOperator({
    (cre_a(3), des_a(3), cre_b(3), des_b(3)): 2.7604767983823093,
    (cre_b(3), des_b(3), cre_a(3), des_a(3)): 2.7604767983823093,
    (cre_a(3), des_a(3), cre_a(0), des_a(0)): 0.34500143431695757,
    (cre_a(3), des_a(2)): 0,
    (cre_b(1), des_b(1), cre_b(1), des_b(1)): 3.629557596782749,
    (cre_b(3), des_b(3), cre_a(1), des_a(1)): 2.007555315292866,
    (cre_b(2), des_b(2), cre_a(0), des_a(0)): 0.03647589722284702,
    (cre_b(0), des_b(0), cre_a(3), des_a(3)): 0.34500143431695757,
    (cre_a(1), des_a(1), cre_a(0), des_a(0)): 1.6873247879821152,
    (cre_a(2), des_a(3)): 0,
    (cre_b(1), des_b(1), cre_b(2), des_b(2)): 0.6903139080585162,
    (cre_b(2), des_b(2), cre_b(0), des_b(0)): 0.03647589722284702,
    (cre_a(3), des_a(3), cre_a(2), des_a(2)): 0.6260185180496143,
    (cre_b(1), des_b(1)): 0.0015941870439207806,
    (cre_b(0), des_b(0), cre_b(2), des_b(2)): 0.03647589722284702,
    (cre_b(3), des_b(2)): 0,
    (cre_b(2), des_b(2), cre_a(1), des_a(1)): 0.690

In [20]:
op_linop = ffsim.linear_operator(op, norb=norb, nelec=nelec)
expval_fermion_op = np.vdot(psi, op_linop @ psi)

np.testing.assert_allclose(expval_fermion_op, expval_linop, rtol=1e-10, atol=1e-10)
print(expval_fermion_op)

(22.585543465564925-8.881784197001252e-16j)


Expectation values for multiple states

If you need expectation values for many state vectors $\{𝜓^{(𝑚)}\}$, you can reuse the same `LinearOperator` and loop.

For diagonal operators, you can reuse the diagonal vector and compute
$\sum_k 𝑑_𝑘 ∣𝜓{_𝑘 ^{(m)}}∣^2$ very easily.

In [21]:
# Example: compute expectations for a small batch of random states
num_states = 5
psis = [ffsim.random.random_state_vector(dim, seed=rng) for _ in range(num_states)]
psis = [v / np.linalg.norm(v) for v in psis]

expvals_linop = [np.vdot(v, H_linop @ v) for v in psis]
expvals_diag = [np.vdot(v, diag * v) for v in psis]

np.testing.assert_allclose(expvals_linop, expvals_diag, rtol=1e-10, atol=1e-10)
print(expvals_linop)

[(23.755155173674137-2.220446049250313e-16j), (23.062138883911075-2.220446049250313e-16j), (24.112302225581004-4.440892098500626e-16j), (24.522358242260772+4.996003610813204e-16j), (25.07589796996848+2.220446049250313e-16j)]


**Summary**

Use `ffsim.linear_operator(op, norb, nelec)` + `np.vdot(psi, op @ psi)` for the general case.

If the operator provides a diagonal via `ffsim.diag`, computing $⟨𝑂⟩$ becomes a cheap weighted sum of probabilities.

The same workflow applies whether `op` is a Hamiltonian object or a `FermionOperator`.